In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from cityseer.metrics import networks
from cityseer.tools import graphs, io

streets_gpd = gpd.read_file("data/madrid_streets/street_network.gpkg")
streets_gpd = streets_gpd.explode(reset_index=True)
G = io.nx_from_generic_geopandas(streets_gpd)
G_dual = graphs.nx_to_dual(G)

100%|██████████| 47155/47155 [00:03<00:00, 14751.65it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 47129/47129 [00:00<00:00, 184526.57it/s]
INFO:cityseer.tools.graphs:Converting graph to dual.
INFO:cityseer.tools.graphs:Preparing dual nodes
100%|██████████| 47129/47129 [00:00<00:00, 116269.42it/s]
INFO:cityseer.tools.graphs:Preparing dual edges (splitting and welding geoms)
100%|██████████| 47129/47129 [00:19<00:00, 2452.28it/s]


In [2]:
# prepare the data structures
nodes_gdf, edges_gdf, network_structure = io.network_structure_from_nx(
    G_dual, crs=streets_gpd.crs
)

INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.
100%|██████████| 47129/47129 [00:16<00:00, 2789.72it/s]


In [3]:
nodes_gdf, edges_gdf, network_structure, stops, avg_stop_pairs = io.add_transport_gtfs(
    "data/madrid_gtfs/madrid_metro",
    nodes_gdf,
    edges_gdf,
    network_structure,
    streets_gpd.crs.to_epsg(),
)


INFO:cityseer.tools.io:Loading GTFS data from data/madrid_gtfs/madrid_metro
INFO:cityseer.tools.io:Loaded 1272 stops and 2360 stop times
INFO:cityseer.tools.io:Adding GTFS stops to network nodes.
1272it [03:26,  6.17it/s]
INFO:cityseer.tools.io:Generating segment durations between stops.
INFO:cityseer.tools.io:Adding GTFS segments to network edges.
279it [00:21, 12.91it/s]


In [4]:
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,avg_wait_time
0,gtfs-data/madrid_gtfs/madrid_metro-par_4_1,1,PLAZA DE CASTILLA,Paseo de la Castellana 189,40.46690,-3.68917,A,http://www.crtm.es,0,est_90_21,NaN,0,430.357143
1,gtfs-data/madrid_gtfs/madrid_metro-acc_4_1_1,1,Plaza de Castilla,Paseo de la Castellana 189,40.46682,-3.68918,NaN,http://www.crtm.es,2,est_90_21,NaN,0,NaN
2,gtfs-data/madrid_gtfs/madrid_metro-acc_4_1_1040,1,Ascensor,Plaza de Castilla 9,40.46555,-3.68877,NaN,http://www.crtm.es,2,est_90_21,NaN,0,NaN
3,gtfs-data/madrid_gtfs/madrid_metro-acc_4_1_1043,1,Intercambiador Superficie,Paseo de la Castellana 191 B,40.46728,-3.68915,NaN,http://www.crtm.es,2,est_90_21,NaN,0,NaN
4,gtfs-data/madrid_gtfs/madrid_metro-acc_4_1_1044,1,Ascensor,Paseo de la Castellana 189,40.46702,-3.68918,NaN,http://www.crtm.es,2,est_90_21,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,gtfs-data/madrid_gtfs/madrid_metro-est_4_234,234,HOSPITAL SEVERO OCHOA,Avda Orellana 3,40.32177,-3.76797,B1,http://www.crtm.es,1,NaN,Europe/Madrid,2,NaN
1268,gtfs-data/madrid_gtfs/madrid_metro-est_4_235,235,LEGANES CENTRAL,Calle Virgen del Camino 1,40.32899,-3.77154,B1,http://www.crtm.es,1,NaN,Europe/Madrid,2,NaN
1269,gtfs-data/madrid_gtfs/madrid_metro-est_4_236,236,SAN NICASIO,Avda Mar Mediterráneo SN,40.33616,-3.77587,B1,http://www.crtm.es,1,NaN,Europe/Madrid,2,NaN
1270,gtfs-data/madrid_gtfs/madrid_metro-par_4_237,237,OPERA,Plaza de Isabel II 9,40.41809,-3.70928,A,http://www.crtm.es,0,est_4_36,NaN,0,13.214286


In [ ]:
# convert stops to geopandas
stops_gdf = gpd.GeoDataFrame(
    stops,
    geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
    crs=4326,  # Adjust the CRS to suit your data if needed
)
stops_gdf = stops_gdf.to_crs(streets_gpd.crs.to_epsg())
stops_gdf

In [ ]:
stops.avg_wait_time.describe()
# count nan
stops.avg_wait_time.isna().sum()
# impute

In [ ]:
avg_stop_pairs

## Calculate centralities

The centrality methods can now be computed.


In [ ]:
distances = [500, 2000, 5000]
nodes_gdf = networks.node_centrality_shortest(
    network_structure=network_structure,
    nodes_gdf=nodes_gdf,
    distances=distances,
)

In [ ]:
nodes_gdf.columns

## Plots

Let's plot a selection of distance thresholds for each of the computed measures.


In [ ]:
import matplotlib.colors as colors

fig, axes = plt.subplots(2, 1, figsize=(8, 12), facecolor="#111")
fig.suptitle(
    "Shortest path centrality",
    fontsize=10,
    color="#ddd",
)

nodes_gdf.plot(
    column="cc_harmonic_2000",
    cmap="magma",
    legend=False,
    ax=axes[0],
)
axes[0].set_title(
    "Harmonic closeness centrality",
    fontsize=8,
    color="#ddd",
)
stops_gdf.plot(ax=axes[0], color="red", markersize=1)

nodes_gdf.plot(
    column="cc_betweenness_2000",
    cmap="magma",
    norm=colors.LogNorm(),  # Apply log normalization
    legend=False,
    ax=axes[1],
)
axes[1].set_title(
    "Betweenness centrality",
    fontsize=8,
    color="#ddd",
)
stops_gdf.plot(ax=axes[1], color="red", markersize=1)

for ax in axes:
    ax.set_xlim(438500, 438500 + 3500)
    ax.set_ylim(4472500, 4472500 + 3500)
    ax.axis(False)

plt.tight_layout()